# Post-process ADM1 summary results

For loading in oi-risk-vis interface.

In [ ]:
import os
from collections import defaultdict
from glob import glob

import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
cyclone_fnames = glob('../results/*/Cyclone/*.csv')
fluvial_fnames = glob('../results/*/Fluvial/*/*.csv')
coastal_fnames = glob('../results/*/Coastal/*/*.csv')

In [ ]:
def tidy_df(df, id_vars, new_id_vars):
    # melt to get a long dataframe with "variable" and "value"
    # columns, where "variable" is taken from each input column name
    df_tidy = df \
        .reset_index() \
        .melt(id_vars=id_vars)
    
    # create a column from each new_id_var 
    # by splitting the "variable" names on "_"
    for i, varname in enumerate(new_id_vars):
        df_tidy[varname] = df_tidy.variable.apply(lambda d: d.split("_")[i+1])
        
    # keep only the first element as the actual variable name 
    df_tidy['variable'] = df_tidy.variable.apply(lambda d: d.split("_")[0])
    
    # parse an integer from return period, if present
    if 'rp' in new_id_vars:
        df_tidy.rp = df_tidy.rp.apply(lambda d: int(d.replace("RP", "")))
        
    # pivot back out to columns for each variable
    df_tidy = df_tidy.pivot_table(index=(id_vars + new_id_vars), columns='variable')
    df_tidy.columns = [v for _, v in df_tidy.columns]
    return df_tidy.reset_index()

In [ ]:
def split_df_by_rp_columns(df):
    df_rp = df[[c for c in df.columns if 'RP' in c]]
    df_exp = df[[c for c in df.columns if 'RP' not in c]]
    return df_rp, df_exp

In [ ]:
def split_regroup(df):
    rp, exp = split_df_by_rp_columns(df)
    exp_tidy = tidy_df(
        exp, 
        id_vars=['region_name', 'sector', 'rcp'],
        new_id_vars=['gcm', 'epoch'])
    rp_tidy = tidy_df(
        rp, 
        id_vars=['region_name', 'sector', 'rcp'],
        new_id_vars=['gcm', 'epoch', 'rp'])
    return rp_tidy, exp_tidy

## Cyclone

In [ ]:
dfs = []
for fname in tqdm(cyclone_fnames):    
    sector = os.path.basename(os.path.dirname(os.path.dirname(fname)))
    df = pd.read_csv(fname)
    df['sector'] = sector
    dfs.append(df)
cyclone = pd.concat(dfs).set_index(['sector','region_name'])

In [ ]:
cyclone_rp, cyclone_exp = split_df_by_rp_columns(cyclone)

cyclone_rp = tidy_df(
    cyclone_rp, 
    id_vars=['region_name', 'sector'], 
    new_id_vars=['hazard', 'rp'])
cyclone_rp['rcp'] = 'historical'
cyclone_rp['gcm'] = 'none'
cyclone_rp['epoch'] = 'HIST'

cyclone_exp = tidy_df(
    cyclone_exp, 
    id_vars=['region_name', 'sector'], 
    new_id_vars=['hazard'])
cyclone_exp['rcp'] = 'historical'
cyclone_exp['gcm'] = 'none'
cyclone_exp['epoch'] = 'HIST'

## Flooding

In [ ]:
dfs = defaultdict(list)
for fname in tqdm(fluvial_fnames):
    climate = os.path.basename(os.path.dirname(fname))
    sector = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(fname))))
    df = pd.read_csv(fname)
    df['sector'] = sector
    df['rcp'] = climate
    dfs[climate].append(df)
fluvial_hist = pd.concat(dfs['historical']).set_index(['region_name','sector','rcp'])
fluvial_fut  = pd.concat(dfs['rcp4p5'] + dfs['rcp8p5']).set_index(['region_name','sector','rcp'])

In [ ]:
fluvial_fut_rp, fluvial_fut_exp = split_regroup(fluvial_fut)

In [ ]:
fluvial_hist_rp, fluvial_hist_exp = split_regroup(fluvial_hist)

In [ ]:
fluvial_rp = fluvial_hist_rp.append(fluvial_fut_rp)
fluvial_rp.epoch.replace('1980', 'HIST', inplace=True) 
fluvial_rp['hazard'] = 'fluvial'

In [ ]:
fluvial_exp = fluvial_hist_exp.append(fluvial_fut_exp)
fluvial_exp.epoch.replace('1980', 'HIST', inplace=True) 
fluvial_exp['hazard'] = 'fluvial'

## Coastal

In [ ]:
dfs = defaultdict(list)
for fname in tqdm(coastal_fnames):
    climate = os.path.basename(os.path.dirname(fname))
    sector = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(fname))))
    df = pd.read_csv(fname)
    df['sector'] = sector
    df['rcp'] = climate
    dfs[climate].append(df)
coastal_hist = pd.concat(dfs['historical']).set_index(['region_name','sector','rcp'])
coastal_fut  = pd.concat(dfs['rcp4p5_perc_50'] + dfs['rcp8p5_perc_50']).set_index(['region_name','sector','rcp'])

In [ ]:
coastal_fut_rp, coastal_fut_exp = split_regroup(coastal_fut)

In [ ]:
coastal_hist_rp, coastal_hist_exp = split_regroup(coastal_hist)
coastal_hist_rp = coastal_hist_rp[coastal_hist_rp.epoch == 'HIST']
coastal_hist_exp = coastal_hist_exp[coastal_hist_exp.epoch == 'HIST']

In [ ]:
coastal_rp = coastal_hist_rp.append(coastal_fut_rp)
# above processing coded the subsidence scenario in the GCM column
# pick only "WS" - with subsidence
coastal_rp = coastal_rp[coastal_rp.gcm == 'WS'].copy()
coastal_rp.gcm = 'none'
coastal_rp.rcp = coastal_rp.rcp.str.replace('_perc_50', '')
coastal_rp['hazard'] = 'coastal'

In [ ]:
coastal_exp = coastal_hist_exp.append(coastal_fut_exp)
# above processing coded the subsidence scenario in the GCM column
# pick only "WS" - with subsidence
coastal_exp = coastal_exp[coastal_exp.gcm == 'WS'].copy()
coastal_exp.gcm = 'none'
coastal_exp.rcp = coastal_exp.rcp.str.replace('_perc_50', '')
coastal_exp['hazard'] = 'coastal'

## Combined

In [ ]:
exp = pd.concat([cyclone_exp, fluvial_exp, coastal_exp])

In [ ]:
rp = pd.concat([cyclone_rp, fluvial_rp, coastal_rp])

In [ ]:
exp.to_csv('../results/expected_all.csv')

In [ ]:
rp.to_csv('../results/rp_all.csv')

Some sense-checking of numbers of results

In [ ]:
pd.set_option("max_rows", 400)

In [ ]:
len(rp.region_name.unique()) # total regions should be 312

In [ ]:
rp.groupby(['hazard', 'rp']).count().reset_index()[['hazard', 'rp']].groupby('hazard').count()

In [ ]:
rp.groupby(['hazard', 'sector', 'rcp', 'gcm', 'epoch']).count()

In [ ]:
2940 / 10, 8736 / 28, 2808 / 9  # 294 is okay for coastal, excludes LAO regions

In [ ]:
exp.groupby(['hazard', 'sector', 'rcp', 'gcm', 'epoch']).count()

Historical - one row per region, sum over sectors and hazards

In [ ]:
hist_exp = exp[exp.epoch=='HIST'] \
    .drop(
        columns=[
            'rcp', 'gcm', 'epoch', 'hazard', 
            'EAEL-primary-gdp', 'EAEL-secondary-gdp', 'EAEL-tertiary-gdp'
        ]) \
    .groupby(['region_name']) \
    .sum()

In [ ]:
fut_exp = exp[exp.epoch=='2080'] \
    .groupby(['region_name', 'sector', 'hazard', 'rcp']) \
    .max() \
    .reset_index() \
    .drop(
        columns=[
            'gcm', 'epoch', 'hazard', 
            'EAEL-primary-gdp', 'EAEL-secondary-gdp', 'EAEL-tertiary-gdp'
        ]) \
    .groupby(['region_name', 'rcp']) \
    .sum() \
    .reset_index() \
    .pivot(index='region_name', columns='rcp', values=['EAEL-gdp','expectedLengthDamaged','maxEAD','minEAD'])
fut_exp.columns = [f"{col}_{rcp}" for col, rcp in fut_exp.columns]

In [ ]:
all_exp = hist_exp.merge(fut_exp, on='region_name')
all_exp.index.name = 'GID_1'
all_exp

In [ ]:
all_exp.to_csv('../results/expected_summary.csv')
len(all_exp)

prob-loss direct/indirect - historical

In [ ]:
def region_summary(rp, region_name):
    region_rp = rp[rp.region_name == region_name]
    region_rp = region_rp[region_rp.epoch == 'HIST']
    region_rp = region_rp[region_rp.rp.isin([10,50,100,500,1000])]
    region_rp = region_rp[['rp', 'assetDamage', 'gdp']] \
        .groupby('rp') \
        .sum() \
        .reset_index()
    # account for disruption
    region_rp.gdp = region_rp.gdp * 30 / 365
    
    region_rp = region_rp \
        .rename(columns={
            'gdp': '1. Economic impact',
            'assetDamage': '2. Direct damages',
        }) \
        .melt(id_vars='rp', var_name='field')
    region_rp['probability'] = 1 / region_rp.rp
    return region_rp.drop(columns='rp') \
        .sort_values(by=['field', 'probability'])

In [ ]:
for region_name in tqdm(rp.region_name.unique()):
    region_summary(rp, region_name).to_csv(f"../results/historical_total/historical_total_{region_name}.csv")

In [ ]:
region_name = 'VNM.9_1'

In [ ]:
def region_breakdown(rp, region_name):
    region_rp = rp[rp.region_name == region_name].copy()
    # account for disruption
    region_rp.gdp = region_rp.gdp * 30 / 365
    region_rp['probability'] = 1 / region_rp.rp

    region_rp = region_rp.drop(columns=['region_name', 'primary-gdp', 'secondary-gdp', 'tertiary-gdp', 'rp', 'gcm'])
    region_rp = region_rp.groupby(['sector', 'hazard', 'rcp', 'epoch', 'probability']) \
        .agg(['min', 'max'])
    region_rp.columns = [f"{var}_{agg}" for var, agg in region_rp.columns]
    return region_rp

In [ ]:
for region_name in tqdm(rp.region_name.unique()):
    region_breakdown(rp, region_name).to_csv(f"../results/regional_breakdown/regional_breakdown_{region_name}.csv")

In [ ]:
def regional_total_by_climate(rp, region_name):
    region_rp = rp[rp.region_name == region_name].copy()
    # account for disruption
    region_rp.gdp = region_rp.gdp * 30 / 365
    region_rp['probability'] = 1 / region_rp.rp
    region_rp = region_rp[region_rp.rp.isin([10,50,100,500,1000])]
    region_rp = region_rp[region_rp.epoch.isin(['HIST', '2080'])]
    region_rp = region_rp.drop(columns=['region_name', 'primary-gdp', 'secondary-gdp', 'tertiary-gdp', 'rp', 'gcm'])
    region_rp = region_rp.groupby(['sector', 'hazard', 'rcp', 'epoch', 'probability']).max().reset_index()
    region_rp = region_rp.groupby(['hazard', 'rcp', 'epoch', 'probability']).sum()
    region_rp['totalDamage'] = region_rp.assetDamage + region_rp.gdp
    return region_rp

In [ ]:
for region_name in tqdm(rp.region_name.unique()):
    regional_total_by_climate(rp, region_name).to_csv(f"../results/climate_total/climate_total_{region_name}.csv")

In [ ]:
regional_total_by_climate(rp, region_name)